In [18]:
# Inputs

INPUT_FILE = 'datas/Borsdata_2022-04-03.xlsx'

MONEY_TO_USE = 350_000
NO_COMPS_TO_BUY = 20
MAX_NO_COMPS_FIRST_NORTH = 5

In [19]:
# Import pandas and ignore warnings

import pandas as pd
pd.set_option('mode.chained_assignment', None)

In [20]:
# Read Excel file and sort by company name

df = pd.read_excel(INPUT_FILE,header=[0],index_col=1,sheet_name='Export')
df.sort_index(inplace=True)

In [21]:
# Apply filter: enterprise value > 1000 MSEK with positive momentums and trends

ev1000 = df['Börsvärde - Senaste'] > 1000
utv3 = df['Kursutveck. - Utveck.  3m'] > 0
utv6 = df['Kursutveck. - Utveck.  6m'] > 0
utv12 = df['Kursutveck. - Utveck.  1år'] > 0
utv6vs3 = df['Kursutveck. - Utveck.  6m'] > df['Kursutveck. - Utveck.  3m']
utv12vs6 = df['Kursutveck. - Utveck.  1år'] > df['Kursutveck. - Utveck.  6m']
ma50 = df['Kurs / MA - MA 50d'] > 0
ma200 = df['Kurs / MA - MA 200d'] > 0
ma200vs50 = df['Kurs / MA - MA 200d'] > df['Kurs / MA - MA 50d']

df_filt = df[ev1000 & utv3 & utv6 & utv12 & utv6vs3 & utv12vs6 & ma50 & ma200 & ma200vs50]

In [22]:
# Add momentum ranks and sort by total rank

header = ['Momentum - Rank 3m','Momentum - Rank 6m','Momentum - Rank 1år','Momentum - Tot']
header2 = ['Kursutveck. - Utveck.  3m','Kursutveck. - Utveck.  6m','Kursutveck. - Utveck.  1år']

for i in range(3):
    df_filt[header[i]] = df_filt[header2[i]].rank(ascending=False,method='first').astype(int)
df_filt[header[3]] = (df_filt[header[0]] + df_filt[header[1]] + df_filt[header[2]]).rank(ascending=True,method='first').astype(int)
df_filt.sort_values(by=[header[3]],inplace=True)

In [23]:
# Display amount of stocks to buy for each company

money_per_stock = MONEY_TO_USE / NO_COMPS_TO_BUY
df_filt['Antal aktier'] = round(money_per_stock / df_filt['Aktiekurs - Senaste']).astype(int)
df_filt.head(NO_COMPS_TO_BUY).sort_index()

,Börsdata ID,Info - Ticker,Info - Lista,Börsvärde - Senaste,Aktiekurs - Senaste,Kursutveck. - Utveck. 3m,Kursutveck. - Utveck. 6m,Kursutveck. - Utveck. 1år,Kurs / MA - MA 50d,Kurs / MA - MA 200d,Info - Aktiekurs,Momentum - Rank 3m,Momentum - Rank 6m,Momentum - Rank 1år,Momentum - Tot,Antal aktier
Bolagsnamn,,,,,,,,,,,,,,,,
Africa Oil,258,AOI,Mid Cap,8187.740,17.160,0.32407,0.34377,1.13035,0.02246,0.30536,2022-04-01,6,9,6,7,1020
Arctic Paper,381,ARP,Small Cap,1548.587,22.350,0.20032,0.46270,0.57394,0.10414,0.27838,2022-04-01,10,8,10,9,783
AstraZeneca,18,AZN,Large Cap,1922872.000,1241.000,0.16767,0.19327,0.42841,0.08901,0.17569,2022-04-01,11,13,14,13,14
B3 Consulting,1313,B3,Small Cap,1016.612,118.500,0.43116,0.76866,1.44330,0.22943,0.56288,2022-04-01,4,4,5,3,148
BE Group,27,BEGR,Small Cap,2333.994,179.400,0.35396,0.72500,2.08247,0.27182,0.59640,2022-04-01,5,6,3,4,98
Filo Mining,1361,FIL,First North,18457.211,152.000,0.74713,1.45955,6.30769,0.30802,0.79755,2022-04-01,2,1,1,1,115
International Petroleum,1443,IPCO,Mid Cap,13496.360,89.400,0.78800,0.95795,2.14346,0.24862,0.71735,2022-04-01,1,2,2,2,196
Josemaria Resources,378,JOSE,Mid Cap,5057.158,13.200,0.25237,0.82825,1.62425,0.05166,0.46199,2022-04-01,8,3,4,5,1326
Lundin Energy,132,LUNE,Large Cap,116057.000,405.900,0.25085,0.26017,0.48355,0.11593,0.24472,2022-04-01,9,10,13,11,43


In [24]:
# Display number of companies from each list

stock_lists = ['Large Cap', 'Mid Cap', 'Small Cap', 'First North']
stock_lists_dict = {}

for stock_list in stock_lists:
    stock_lists_dict[stock_list] = sum(df_filt.head(NO_COMPS_TO_BUY)['Info - Lista'] == stock_list)

for stock_list,num in stock_lists_dict.items():
    print(f'{stock_list:11} {num}')

print(f'Total       {sum(stock_lists_dict.values())}')

Large Cap   4
Mid Cap     3
Small Cap   5
First North 3
Total       15


In [25]:
# Reduce number of companies from First North

df_filt_fn = pd.concat([df_filt[df_filt['Info - Lista'] != 'First North'].head(NO_COMPS_TO_BUY), df_filt[df_filt['Info - Lista'] == 'First North'].head(MAX_NO_COMPS_FIRST_NORTH)])
df_filt_fn.sort_values(by=[header[3]],inplace=True)

In [26]:
# Display amount of stocks to buy for each company

df_filt_fn['Antal aktier'] = round(money_per_stock / df_filt_fn['Aktiekurs - Senaste']).astype(int)
df_filt_fn.head(NO_COMPS_TO_BUY).sort_index()

,Börsdata ID,Info - Ticker,Info - Lista,Börsvärde - Senaste,Aktiekurs - Senaste,Kursutveck. - Utveck. 3m,Kursutveck. - Utveck. 6m,Kursutveck. - Utveck. 1år,Kurs / MA - MA 50d,Kurs / MA - MA 200d,Info - Aktiekurs,Momentum - Rank 3m,Momentum - Rank 6m,Momentum - Rank 1år,Momentum - Tot,Antal aktier
Bolagsnamn,,,,,,,,,,,,,,,,
Africa Oil,258,AOI,Mid Cap,8187.740,17.160,0.32407,0.34377,1.13035,0.02246,0.30536,2022-04-01,6,9,6,7,1020
Arctic Paper,381,ARP,Small Cap,1548.587,22.350,0.20032,0.46270,0.57394,0.10414,0.27838,2022-04-01,10,8,10,9,783
AstraZeneca,18,AZN,Large Cap,1922872.000,1241.000,0.16767,0.19327,0.42841,0.08901,0.17569,2022-04-01,11,13,14,13,14
B3 Consulting,1313,B3,Small Cap,1016.612,118.500,0.43116,0.76866,1.44330,0.22943,0.56288,2022-04-01,4,4,5,3,148
BE Group,27,BEGR,Small Cap,2333.994,179.400,0.35396,0.72500,2.08247,0.27182,0.59640,2022-04-01,5,6,3,4,98
Filo Mining,1361,FIL,First North,18457.211,152.000,0.74713,1.45955,6.30769,0.30802,0.79755,2022-04-01,2,1,1,1,115
International Petroleum,1443,IPCO,Mid Cap,13496.360,89.400,0.78800,0.95795,2.14346,0.24862,0.71735,2022-04-01,1,2,2,2,196
Josemaria Resources,378,JOSE,Mid Cap,5057.158,13.200,0.25237,0.82825,1.62425,0.05166,0.46199,2022-04-01,8,3,4,5,1326
Lundin Energy,132,LUNE,Large Cap,116057.000,405.900,0.25085,0.26017,0.48355,0.11593,0.24472,2022-04-01,9,10,13,11,43


In [27]:
# Display number of companies from each list (First North reduced)

stock_lists = ['Large Cap', 'Mid Cap', 'Small Cap', 'First North']
stock_lists_dict = {}

for stock_list in stock_lists:
    stock_lists_dict[stock_list] = sum(df_filt_fn.head(NO_COMPS_TO_BUY)['Info - Lista'] == stock_list)

for stock_list,num in stock_lists_dict.items():
    print(f'{stock_list:11} {num}')
    
print(f'Total       {sum(stock_lists_dict.values())}')

Large Cap   4
Mid Cap     3
Small Cap   5
First North 3
Total       15
